README: https://docs.google.com/document/d/1cxqU0d6jLgmjK3pTIK_nI7M9fo7NDUt3qGktD8knNZ8/edit  

API Details:  
https://min-api.cryptocompare.com/  
https://www.cryptocompare.com/api  

Code Ref:  
https://github.com/MaverickLLC/cryptocompareapi_python_MavericLLC/blob/master/cryptocompare_maverick.py  
https://github.com/lagerfeuer/cryptocompare/blob/master/cryptocompare/cryptocompare.py  

TO DO:  
- Make generic functions for tasks for modularity
- Make a class for individual coin if required
- Incorporate https://github.com/mrjbq7/ta-lib
- Notifiers https://github.com/AbenezerMamo/crypto-signal/tree/master/app/notifiers

In [1]:
# Imports
import time
import logging
import requests
import datetime

import pandas as pd

In [2]:
# API
API_ENDPOINT = "https://min-api.cryptocompare.com/data"
OLD_API_ENDPOINT = "https://www.cryptocompare.com/api/data"

URL_COIN_LIST = OLD_API_ENDPOINT + "/coinlist/"
URL_COIN_SNAPSHOT = OLD_API_ENDPOINT + "/coinsnapshot/?fsym={}&tsym={}&e={}"

URL_PRICE = API_ENDPOINT + "/pricemulti?fsyms={}&tsyms={}&e={}"
URL_PRICE_MULTI = API_ENDPOINT + "/pricemulti?fsyms={}&tsyms={}&e={}"
URL_PRICE_MULTI_FULL = API_ENDPOINT + "/pricemultifull?fsyms={}&tsyms={}&e={}"

URL_HIST_PRICE = API_ENDPOINT + "/pricehistorical?fsym={}&tsyms={}&ts={}"
URL_HIST_PRICE_DAY = API_ENDPOINT + "/histoday?fsym={}&tsym={}&e={}&limit={}&allData={}"
URL_HIST_PRICE_HOUR = API_ENDPOINT + "/histohour?fsym={}&tsym={}&e={}&limit={}"
URL_HIST_PRICE_MINUTE = API_ENDPOINT + "/histominute?fsym={}&tsym={}&e={}&toTs={}"

URL_AVG = API_ENDPOINT + "/generateAvg?fsym={}&tsym={}&e={}"
URL_DAY_AVG = API_ENDPOINT + "/dayAvg?fsym={}&tsym={}&e={}"

In [3]:
# FIELDS
PRICE = 'PRICE'
HIGH = 'HIGH24HOUR'
LOW = 'LOW24HOUR'
VOLUME = 'VOLUME24HOUR'
CHANGE = 'CHANGE24HOUR'
CHANGE_PERCENT = 'CHANGEPCT24HOUR'
MARKETCAP = 'MKTCAP'

In [4]:
# Defaults
CURR = "USD"
EXCHANGE = "CCCAGG"
COIN = 'BTC'
COIN_LIST = ['BTC', 'ETH']

In [5]:
def query_cryptocompare(url):
    """ Query CryptoCompare API """
    try:
        response = requests.get(url).json()
    except Exception as e:
        logging.error("Failure while querying {query}. \n{err}".format(query=url, err=e))
        return None
    return response

In [6]:
# Helper functions
def format_parameter(parameter):
    """ Format parameters for the query """
    if isinstance(parameter, list):
        return ','.join(parameter)
    else:
        return parameter
    
def convert_timestamp(timestamp):
    return datetime.datetime.fromtimestamp(int(timestamp)).strftime('%Y-%m-%d %H:%M:%S')

def get_data(response):
    """ Separate query data from response """
    header = {key: (value if key != 'Data' else len(value)) for key, value in response.items()}
    data = response['Data']
    return header, data

def get_readable_df(response):
    """ Convert timestamp into readable datetime """
    header, data = get_data(response)
    df_data = pd.DataFrame(data)
    df_data = df_data.rename(columns={'time': 'timestamp'})
    df_data['time'] = df_data.timestamp.apply(convert_timestamp)
    df_data = df_data.set_index('time')
    del df_data['timestamp']
    return df_data

In [7]:
def get_coin_list():
    """ Get coin list """
    return query_cryptocompare(URL_COIN_LIST)['Data']

In [8]:
coins = get_coin_list()
COIN_DB = pd.DataFrame.from_dict(coins, orient='index')

In [9]:
def get_price(coin, to_curr=CURR, exchange=EXCHANGE):
    """ Get real time price of the coin """
    if isinstance(coin, list):
        return query_cryptocompare(URL_PRICE_MULTI.format(
                format_parameter(coin), format_parameter(to_curr), exchange
            )
        )
    else:
        return query_cryptocompare(URL_PRICE.format(coin, format_parameter(to_curr), exchange))

In [10]:
get_price(COIN_LIST)

{'BTC': {'USD': 11447.73}, 'ETH': {'USD': 947.17}}

In [11]:
def get_historical_price_timestamp(coin, to_curr=CURR, timestamp=time.time(), exchange=EXCHANGE):
    """ Get value of coin in currency at a particular timestamp """
    if isinstance(timestamp, datetime.datetime):
        timestamp = time.mktime(timestamp.timetuple())
    
    return query_cryptocompare(URL_HIST_PRICE.format(
            coin, 
            format_parameter(to_curr),
            int(timestamp),
            exchange
        )
    )

In [12]:
get_historical_price_timestamp(COIN)

{'BTC': {'USD': 11171.63}}

In [13]:
def get_historical_price_day(coin, to_curr=CURR, timestamp=time.time(), exchange=EXCHANGE, limit=30, allData='false'):
    """ Get price per day for the past month """
    return query_cryptocompare(URL_HIST_PRICE_DAY.format(
            coin,
            format_parameter(to_curr),
            exchange,
            limit,
            allData
        )
    )

def get_historical_price_last_day(*args):
    """ Get price for last day """
    return get_historical_price_day(*args, limit=1)

In [14]:
coin_day = get_historical_price_day(COIN)
df_coin_day = get_readable_df(coin_day)

In [15]:
def get_historical_price_day_full(*args):
    """ Get price per day for all time  """
    return get_historical_price_day(*args, allData='true')

In [16]:
coin_day_full = get_historical_price_day_full(COIN)
df_coin_day_full = get_readable_df(coin_day_full)

In [17]:
def get_historical_price_hour(coin, to_curr=CURR, exchange=EXCHANGE, limit=168):
    """ Get price per hour for past 7 days """
    return query_cryptocompare(URL_HIST_PRICE_HOUR.format(
            coin,
            format_parameter(to_curr),
            exchange,
            limit
        )
    )

def get_historical_price_last_hour(*args, **kwargs):
    """ Get price for the last hour """
    return get_historical_price_hour(*args, **kwargs, limit=1)

In [18]:
coin_hour = get_historical_price_hour(COIN)
df_coin_hour = get_readable_df(coin_hour)

In [19]:
def get_historical_price_minute(coin, to_curr=CURR, exchange=EXCHANGE, toTs=time.time()):
    """ Get price per min for past 24 hours """
    return query_cryptocompare(URL_HIST_PRICE_MINUTE.format(
            coin,
            format_parameter(to_curr),
            exchange,
            int(toTs)
        )
    )

In [20]:
coin_min = get_historical_price_minute(COIN)
df_coin_min = get_readable_df(coin_min)

In [52]:
def get_historical_price_minute_by_day(*args, days_ago=0):
    """ Get price per min for 24 hours till days_ago """
    if days_ago > 7:
        logging.error("Can not get information by minute for more than 7 days. Getting information for last possible day.")
        days_ago = 7
    days_ago -= 1  # Subtracting one day as 'toTs' considers ending timestamp
    ts = datetime.datetime.today() - datetime.timedelta(days_ago)
    ts = time.mktime(ts.timetuple())
    return get_historical_price_minute(*args, toTs=int(ts))

In [22]:
coin_minute_week = get_historical_price_minute_by_day(COIN, days_ago=7)
df_coin_minute_week = get_readable_df(coin_minute_week)

-----------------------

In [23]:
# Data
csv_day_full = 'coin_day_full.csv'

In [24]:
df_coin_day_full['coin'] = 'BTC'
df_coin_day_full.to_csv(csv_day_full)

In [25]:
def fetch_last_day(coin):
    """ Get data for last day """
    coin_last_day = get_historical_price_last_day(COIN)
    df_coin_last_day = get_readable_df(coin_last_day)
    df_coin_last_day['coin'] = coin
    last_entry = df_coin_last_day.iloc[-1:]
    return last_entry

In [53]:
def update_last_day(coin):
    """ Update csv for last day """
    record_last_day = fetch_last_day(COIN)
    csv_day = 'coin_day_full.csv'
    df_day = pd.read_csv(csv_day)
    record = df_day[df_day['time'] == record_last_day.iloc[0]['time']]
    if record.empty:
        record_last_day.to_csv(csv_day, mode='a', header=False)

In [27]:
from functools import partial
update_last_day_partial = partial(update_last_day, COIN)

In [58]:
from apscheduler.schedulers.background import BackgroundScheduler
scheduler = BackgroundScheduler({'apscheduler.timezone': 'UTC'})
# Update data every day at 5:30pm
scheduler.add_job(update_last_day_partial, 'cron', day_of_week='*', hour=17, minute=30)
scheduler.start()

--------------------------

In [54]:
# Maps csv (future data objects) to period granularity
# If we store all data together in a single data source, we'll change this to a function which returns corresponding rows
data_csv_period_mapping = {
    "day": 'coin_day_full.csv',
    "hour": 'coin_hour.csv',
    "min": 'coin_min.csv'
}

In [50]:
def fetch_data(coin_symbol=['BTC'], nrows=1, period='day'):
    """ Fetch data from csv for mentioned coin 
        This function fetches last nrows from csv corr to given period
    """
    csv_filename = data_csv_period_mapping[period]
    df_csv = pd.read_csv(csv_filename, index_col='time')
    
    # Get rows where 'coin' is one of the coin_symbols; then extract last nrows
    req_data = df_csv[df_csv['coin'].isin(coin_symbol)].iloc[-nrows:]
    req_columns = ['open', 'high', 'low', 'close', 'volumeto']
    req_data = req_data[req_columns]
    return req_data

In [51]:
fetch_data(['BTC'], 10, 'day')

,open,high,low,close,volumeto
time,,,,,
2018-02-10 16:00:00,8569.32,8573.35,7862.31,8084.61,1.013772e+09
2018-02-11 16:00:00,8084.61,8997.34,8084.41,8911.27,1.085922e+09
2018-02-12 16:00:00,8911.17,8955.15,8379.35,8544.69,8.533204e+08
2018-02-13 16:00:00,8544.69,9508.22,8542.98,9485.64,1.220863e+09
2018-02-14 16:00:00,9485.64,10223.58,9363.38,10033.75,1.666668e+09
2018-02-15 16:00:00,10033.75,10303.14,9720.38,10188.73,1.061828e+09
2018-02-16 16:00:00,10188.73,11119.45,10074.07,11097.21,1.327921e+09
2018-02-17 16:00:00,11097.21,11288.34,10161.01,10417.23,1.659704e+09
2018-02-18 16:00:00,10418.12,11265.96,10336.32,11182.28,1.186536e+09
